In [2]:
from parser import DFG_python,DFG_java,DFG_ruby,DFG_go,DFG_php,DFG_javascript,DFG_csharp
from parser import (remove_comments_and_docstrings,
                   tree_to_token_index,
                   index_to_code_token,
                   tree_to_variable_index)
import jsonlines
import bleu
import weighted_ngram_match
import syntax_match
import dataflow_match

In [3]:
dfg_function={
    'python':DFG_python,
    'java':DFG_java,
    'ruby':DFG_ruby,
    'go':DFG_go,
    'php':DFG_php,
    'javascript':DFG_javascript,
    'c_sharp':DFG_csharp,
}

In [37]:
data = []
with jsonlines.open('synthesis_results_.jsonl') as f:
    for i in f:
        data.append(i)

In [ ]:
#CodeX and GPT3.5 
lang = 'python'
alpha,beta,gamma,theta = 0.25,0.25,0.25,0.25
references = [[i['label'].strip()] for i in data]
hypothesis = [i['choices'][0]['text'].strip() for i in data]
tokenized_hyps = [x.split() for x in hypothesis]
tokenized_refs = [[x.split() for x in reference] for reference in references]
ngram_match_score = bleu.corpus_bleu(tokenized_refs,tokenized_hyps)
keywords = [x.strip() for x in open('keywords/'+lang+'.txt', 'r', encoding='utf-8').readlines()]
def make_weights(reference_tokens, key_word_list):
    return {token:1 if token in key_word_list else 0.2 \
            for token in reference_tokens}
tokenized_refs_with_weights = [[[reference_tokens, make_weights(reference_tokens, keywords)]\
            for reference_tokens in reference] for reference in tokenized_refs]
weighted_ngram_match_score = weighted_ngram_match.corpus_bleu(tokenized_refs_with_weights,tokenized_hyps)
syntax_match_score = syntax_match.corpus_syntax_match(references, hypothesis, lang)
dataflow_match_score = dataflow_match.corpus_dataflow_match(references, hypothesis, lang)
code_bleu_score = alpha*ngram_match_score\
                + beta*weighted_ngram_match_score\
                + gamma*syntax_match_score\
                + theta*dataflow_match_score
ref = {}
hypo = {}
count = 0
em=0
for i in data:
    ref[count] = [i['label']]
    hypo[count] = [i['choices'][0]['text'] if i['choices'][0]['text'].strip() is not '' else '.']
    if ' '.join(ref[count][0].strip().split()) == ' '.join(hypo[count][0].strip().split()):
        em+=1
    count+=1
print(round(100*em/len(data),2),round(100*syntax_match_score,2),round(100*dataflow_match_score,2),round(100*code_bleu_score,2))

In [ ]:
#ChatGPT
lang = 'python'
alpha,beta,gamma,theta = 0.25,0.25,0.25,0.25
references = [[i['label'].strip()] for i in data]
hypothesis = [i['choices'][0]['message']['content'].strip() for i in data]
tokenized_hyps = [x.split() for x in hypothesis]
tokenized_refs = [[x.split() for x in reference] for reference in references]
ngram_match_score = bleu.corpus_bleu(tokenized_refs,tokenized_hyps)
keywords = [x.strip() for x in open('keywords/'+lang+'.txt', 'r', encoding='utf-8').readlines()]
def make_weights(reference_tokens, key_word_list):
    return {token:1 if token in key_word_list else 0.2 \
            for token in reference_tokens}
tokenized_refs_with_weights = [[[reference_tokens, make_weights(reference_tokens, keywords)]\
            for reference_tokens in reference] for reference in tokenized_refs]
weighted_ngram_match_score = weighted_ngram_match.corpus_bleu(tokenized_refs_with_weights,tokenized_hyps)
syntax_match_score = syntax_match.corpus_syntax_match(references, hypothesis, lang)
dataflow_match_score = dataflow_match.corpus_dataflow_match(references, hypothesis, lang)
code_bleu_score = alpha*ngram_match_score\
                + beta*weighted_ngram_match_score\
                + gamma*syntax_match_score\
                + theta*dataflow_match_score
ref = {}
hypo = {}
count = 0
em=0
for i in data:
    ref[count] = [i['label']]
    hypo[count] = [i['choices'][0]['message']['content'] if i['choices'][0]['message']['content'].strip() is not '' else '.']
    if ' '.join(ref[count][0].strip().split()) == ' '.join(hypo[count][0].strip().split()):
        em+=1
    count+=1
print(round(100*em/len(data),2),round(100*syntax_match_score,2),round(100*dataflow_match_score,2),round(100*code_bleu_score,2))